# Using __new__()

Metaclass example 1 – ordered attributes



In [10]:
import collections


class OrderedAttributes(type):
    @classmethod
    def __prepare__(mcs, name, bases, **kwargs):
        return collections.OrderedDict()

    def __new__(mcs, name, bases, namespace, **kwargs):
        result = super().__new__(mcs, name, bases, namespace)
        result._ordered_attributes = tuple(
            n for n in namespace if not n.startswith('__'))
        return result

    def __iter__(cls):
        return iter(cls._ordered_attributes)


class OderPreserved(metaclass=OrderedAttributes):
    pass


class Something(OderPreserved):
    this = 'text'

    def z(self):
        return False
    b = 'order is preserved'
    a = 'more text'


Something._ordered_attributes

('this', 'z', 'b', 'a')

Metaclass example 2 – self-reference


In [17]:
class Unit:
    """
    Full name for the unit
    """
    factor = 1.0
    standard = None

    @classmethod
    def value(class_, value):
        return value * class_.factor

    @classmethod
    def convert(class_, value):
        if value is None:
            return None
        return value * class_.factor


class UintMeta(type):
    def __new__(cls, name, bases, dict):
        new_cls = super().__new__(cls, name, bases, dict)
        new_cls.standard = new_cls
        return new_cls


class StandardUnit(Unit, metaclass=UintMeta):
    pass


class INCH(StandardUnit):
    """Inches"""
    standard = 'in'


class FOOT(Unit):
    """Feet"""
    name = "ft"
    standard = INCH
    factor = 1/12


class CENTIMETER(Unit):
    """Centimeters"""
    name = "cm"
    standard = INCH
    factor = 2.54


class METER(Unit):
    """Meters"""
    name = "m"
    standard = INCH
    factor = 0.0254


x_std = INCH.value(100)
print(CENTIMETER.convert(x_std))

254.0



The ABCs of Consistent Design
---

In [5]:
from abc import ABCMeta, abstractmethod


class AbstractBettingStrategy(metaclass=ABCMeta):
    __slots__ = ()

    @abstractmethod
    def bet(self, hand):
        return 1

    @abstractmethod
    def record_win(self, hand):
        pass

    @abstractmethod
    def record_loss(self, hand):
        pass

    @classmethod
    def __subclasshook__(cls, subclass):
        if cls is Hand:
            if (any("bet" in B.__dict__ for B in subclass.__mro__)
                and any("record_win" in B.__dict__ for B in subclass.__mro__)
                and any("record_loss" in B.__dict__ for B in subclass.__mro__)
                ):
                return True
        return NotImplemented


class Simple_Broken(AbstractBettingStrategy):
    def bet(self, hand):
        return 1


class Simple(AbstractBettingStrategy):
    def bet(self, hand):
        return 1

    def record_win(self, hand):
        pass

    def record_loss(self, hand):
        pass


simple = Simple()

Using Callables and Contexts
---

In [2]:
import collections.abc
import timeit

iterative = timeit.timeit("pow1(2,1024)", """

class Power1(collections.abc.Callable):
    def __call__(self, x, n):
        p = 1
        for i in range(n):
            p *= x
        return p


pow1 = Power1()
""", number=100000)


# performance improvement


class Power4(collections.abc.Callable):
    def __call__(self, x, n):
        if n == 0:
            return 1
        elif n % 2 == 1:
            return self.__call__(x, n-1)*x
        else:  # n % 2 == 0:
            t = self.__call__(x, n//2)
            return t*t


pow4 = Power4()
pow4(2, 10)

1024